## Importing necessary libaries

In [10]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as pl
from sklearn.linear_model import LassoCV, RidgeCV
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import MinMaxScaler, StandardScaler

## Load machine failure file

In [11]:
data = pd.read_csv('../machine failure.csv')
data

,UDI,Product ID,Type,Air temperature [K],Process temperature [K],Rotational speed [rpm],Torque [Nm],Tool wear [min],Machine failure,TWF,HDF,PWF,OSF,RNF
0,1,M14860,M,298.1,308.6,1551,42.8,0,0,0,0,0,0,0
1,2,L47181,L,298.2,308.7,1408,46.3,3,0,0,0,0,0,0
2,3,L47182,L,298.1,308.5,1498,49.4,5,0,0,0,0,0,0
3,4,L47183,L,298.2,308.6,1433,39.5,7,0,0,0,0,0,0
4,5,L47184,L,298.2,308.7,1408,40.0,9,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,9996,M24855,M,298.8,308.4,1604,29.5,14,0,0,0,0,0,0
9996,9997,H39410,H,298.9,308.4,1632,31.8,17,0,0,0,0,0,0
9997,9998,M24857,M,299.0,308.6,1645,33.4,22,0,0,0,0,0,0
9998,9999,H39412,H,299.0,308.7,1408,48.5,25,0,0,0,0,0,0


## Exploratory analysis

In [12]:
data = pd.get_dummies(data, columns=['Type', ])
data.head()

,UDI,Product ID,Air temperature [K],Process temperature [K],Rotational speed [rpm],Torque [Nm],Tool wear [min],Machine failure,TWF,HDF,PWF,OSF,RNF,Type_H,Type_L,Type_M
0,1,M14860,298.1,308.6,1551,42.8,0,0,0,0,0,0,0,0,0,1
1,2,L47181,298.2,308.7,1408,46.3,3,0,0,0,0,0,0,0,1,0
2,3,L47182,298.1,308.5,1498,49.4,5,0,0,0,0,0,0,0,1,0
3,4,L47183,298.2,308.6,1433,39.5,7,0,0,0,0,0,0,0,1,0
4,5,L47184,298.2,308.7,1408,40.0,9,0,0,0,0,0,0,0,1,0


In [13]:
data.dtypes

UDI                          int64
Product ID                  object
Air temperature [K]        float64
Process temperature [K]    float64
Rotational speed [rpm]       int64
Torque [Nm]                float64
Tool wear [min]              int64
Machine failure              int64
TWF                          int64
HDF                          int64
PWF                          int64
OSF                          int64
RNF                          int64
Type_H                       uint8
Type_L                       uint8
Type_M                       uint8
dtype: object

## Consolidating types of machine failure into the machine failure column

In [14]:
data['Machine failure'] = np.where((data['TWF'] == 1) | (data['HDF'] == 1) | (data['PWF'] == 1) | (data['OSF'] == 1), 1, 0)

cleaned_data = data.drop(['UDI','Product ID', 'TWF', 'HDF', 'PWF', 'OSF', 'RNF'], axis=1)

cleaned_data

,Air temperature [K],Process temperature [K],Rotational speed [rpm],Torque [Nm],Tool wear [min],Machine failure,Type_H,Type_L,Type_M
0,298.1,308.6,1551,42.8,0,0,0,0,1
1,298.2,308.7,1408,46.3,3,0,0,1,0
2,298.1,308.5,1498,49.4,5,0,0,1,0
3,298.2,308.6,1433,39.5,7,0,0,1,0
4,298.2,308.7,1408,40.0,9,0,0,1,0
...,...,...,...,...,...,...,...,...,...
9995,298.8,308.4,1604,29.5,14,0,0,0,1
9996,298.9,308.4,1632,31.8,17,0,1,0,0
9997,299.0,308.6,1645,33.4,22,0,0,0,1
9998,299.0,308.7,1408,48.5,25,0,1,0,0


## Building Ridge regression model

In [15]:
#Load the predictor and target variables
X = cleaned_data.drop(columns=['Machine failure'])
y = cleaned_data['Machine failure']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Standardize (scale) the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Create a RidgeCV model with alpha optimization using cross-validation
alphas = [0.01, 0.1, 1.0, 10.0]  # List of alpha values to test
ridge = RidgeCV(alphas=alphas, store_cv_values=True)

# Fit the Ridge regression model to the training data
ridge.fit(X_train_scaled, y_train)

# Print the best alpha determined by cross-validation
print("Best alpha:", ridge.alpha_)

# Evaluate the model on the test data
y_pred = ridge.predict(X_test_scaled)
mse = mean_squared_error(y_test, y_pred)
print("Mean Squared Error on Test Data:", mse)

Best alpha: 1.0
Mean Squared Error on Test Data: 0.025101684998604153


## Analyzing feature weights

In [16]:
feature_weights = ridge.coef_
feature_names = X.columns

print("Feature Weights (Coefficients):")
for feature, weight in zip(feature_names, feature_weights):
    print(f"{feature}: {weight:.4f}")

Feature Weights (Coefficients):
Air temperature [K]: 0.0426
Process temperature [K]: -0.0300
Rotational speed [rpm]: 0.0924
Torque [Nm]: 0.1145
Tool wear [min]: 0.0200
Type_H: -0.0040
Type_L: 0.0031
Type_M: -0.0007


## Building Lasso regression model

In [17]:
#Load the predictor and target variables
X = cleaned_data.drop(columns=['Machine failure'])
y = cleaned_data['Machine failure']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Standardize (scale) the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Create a LassoCV model with alpha optimization using cross-validation
alphas = [0.01, 0.1, 1.0, 10.0]  # List of alpha values to test
lasso = LassoCV(alphas=alphas, cv=5)

# Fit the Lasso regression model to the training data
lasso.fit(X_train_scaled, y_train)

# Print the best alpha determined by cross-validation
print("Best alpha:", lasso.alpha_)

# Evaluate the model on the test data
y_pred = lasso.predict(X_test_scaled)
mse = mean_squared_error(y_test, y_pred)
print("Mean Squared Error on Test Data:", mse)

Best alpha: 0.01
Mean Squared Error on Test Data: 0.02709712063763147


## Analyzing feature weights

In [18]:
feature_weights = lasso.coef_
feature_names = X.columns

print("Feature Weights (Coefficients):")
for feature, weight in zip(feature_names, feature_weights):
    print(f"Feature {feature}: {weight:.4f}")

Feature Weights (Coefficients):
Feature Air temperature [K]: 0.0074
Feature Process temperature [K]: 0.0000
Feature Rotational speed [rpm]: 0.0127
Feature Torque [Nm]: 0.0350
Feature Tool wear [min]: 0.0097
Feature Type_H: -0.0000
Feature Type_L: 0.0000
Feature Type_M: -0.0000
